In [37]:
import numpy as np
import math
import ipywidgets as widgets
from ipycanvas import Canvas, hold_canvas
from time import sleep
from colour import Color
from numba import njit
from numba.np.extensions import cross2d

## 常用矩阵变换函数

In [38]:
def rotate_x(vertices, a):
    m = np.array([
        [1, 0, 0, 0],
        [0, math.cos(a), math.sin(a), 0],
        [0, -math.sin(a), math.cos(a), 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def rotate_y(vertices, a):
    m = np.array([
        [math.cos(a), 0, -math.sin(a), 0],
        [0, 1, 0, 0],
        [math.sin(a), 0, math.cos(a), 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def rotate_z(vertices, a):
    m = np.array([
        [math.cos(a), math.sin(a), 0, 0],
        [-math.sin(a), math.cos(a), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def translate(vertices, dx, dy, dz):
    m = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [dx, dy, dz, 1],
    ])
    return vertices @ m

def scale(vertices, sx, sy, sz):
    m = np.array([
        [sx, 0, 0, 0],
        [0, sy, 0, 0],
        [0, 0, sz, 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

@njit(fastmath=True)
def point_in_triangle(p, triangle):
    vs1 = triangle[1:]-triangle[:3]
    vs2 = p - triangle[:3]
    s = np.sign(cross2d(vs1,vs2)) >= 0
    r = np.all(s == s[0])
    return r

@njit(fastmath=True)
def point_dist(p1,p2):
    return np.sum(np.power(p1-p2, 2)) ** 0.5

def hex_to_rgb(h):
    return list((np.array(Color(h).rgb)*255).astype('int'))

def color_alpha(c, a):
    return [*c[:3],a]

def color_overlay(c1, c2):
    r = c2[3]/255.0
    return (1-r)*c2 + r*c1

## 3D可视化场景对象

In [39]:
class Object3D():
    def __init__(self, vertices, facets, facet_colors=None):
        self.vertices = vertices
        self.facets = facets
        self.facet_colors = facet_colors
    
    def move(self, dx, dy, dz):
        self.vertices = self.vertices @ translate(dx, dy, dz)
    def rotate_x(self, d):
        self.vertices = self.vertices @ rotate_x(d)
    def rotate_y(self, d):
        self.vertices = self.vertices @ rotate_y(d)
    def rotate_z(self, d):
        self.vertices = self.vertices @ rotate_z(d)

class Camera:
    def __init__(self, position, heading, pitch, yaw):
        self.position = position
        self.heading = heading
        self.pitch = pitch
        self.yaw = yaw
    def rotate_matrix(self):
        return rotate_z(rotate_x(rotate_y(np.eye(4), -self.heading), -self.pitch), -self.yaw)
    def translate_matrix(self):
        return translate(np.eye(4), -self.position[0], -self.position[1], -self.position[2])
    def view_matrix(self):
        return self.translate_matrix() @ self.rotate_matrix()
    
class Render():
    def __init__(self, height, width, camera:Camera):
        self.height = height
        self.width = width
        self.canvas = Canvas(height=self.height, width=self.width)
        self.objects = []
        self.camera = camera
        self.projection_matrix, self.to_screen_matrix = self.projection()
        self.show_axis = False
        self.frame_buffer = np.repeat(255, self.height*self.width*3).reshape((height,width,3))
        self.depth_buffer = np.repeat(1E9*1.0, self.height*self.width).reshape((height,-1))
    
    def projection(self):
        h_fov = math.pi / 3
        v_fov = h_fov * (self.height / self.width)

        NEAR = 0.1
        FAR = 100
        RIGHT = math.tan(h_fov / 2)
        LEFT = -RIGHT
        TOP = math.tan(v_fov / 2)
        BOTTOM = -TOP

        m00 = 2.0 / (RIGHT - LEFT)
        m11 = 2.0 / (TOP - BOTTOM)
        m22 = (FAR + NEAR)*1.0 / (FAR - NEAR)
        m32 = -2.0 * NEAR * FAR / (FAR - NEAR)

        # 投影
        projection_matrix = np.array([
            [m00, 0, 0, 0],
            [0, m11, 0, 0],
            [0, 0, m22, 1],
            [0, 0, m32, 0]
        ])

        # 视口变换
        HW, HH = self.width/2, self.height/2
        to_screen_matrix = np.array([
            [HW, 0, 0, 0],
            [0, -HH, 0, 0],
            [0, 0, 1, 0],
            [HW, HH, 0, 1]
        ])
        return projection_matrix, to_screen_matrix
    
    def add_object(self, obj:Object3D):
        self.objects.append(obj)
    
    @staticmethod
    def _draw_seg(canvas:Canvas, v1, v2):
        if any(np.isnan(v1)) or any(np.isnan(v2)):
            return
        canvas.stroke_line(
            int(v1[0]), 
            int(v1[1]), 
            int(v2[0]), 
            int(v2[1]))
        
    def _draw_obj(self, obj:Object3D, clip=True):
        # trans
        vertices = obj.vertices
        vertices = vertices @ self.camera.view_matrix()
        vertices = vertices @ self.projection_matrix
        # 这里使用w值归一化
        vertices /= vertices[:, -1].reshape(-1, 1)
        
        if clip:
            vertices[(vertices > 2) | (vertices < -2)] = None
        
        vertices = vertices @ self.to_screen_matrix
        
        if not obj.facet_colors is None:
            for i in range(len(obj.facets)):
                self._fill_facet(obj, vertices, i)
        else:
            for facet in obj.facets:
                for i in range(1, len(facet)):
                    Render._draw_seg(self.canvas, vertices[facet[i-1]], vertices[facet[i]])
    
    # 绘制三角形，并填充颜色
    def _fill_facet(self, obj:Object3D, vertices, iface):
        facet = vertices[obj.facets[iface]]
        facet2D = facet[:,:2]
        color = obj.facet_colors[iface]
        boxMin = np.min(facet, axis=0).astype('int')
        boxMax = np.max(facet, axis=0).astype('int')
        for x in range(boxMin[0], boxMax[0]+1):
            for y in range(boxMin[1], boxMax[1]+1):
                if x<0 or x>=self.width or y<0 or y>=self.height:
                    continue
                p = np.array([x,y])
                if point_in_triangle(p, facet2D):
                    
                    # 重心坐标求Z值
                    A = point_dist(p, facet2D[0])
                    B = point_dist(p, facet2D[1])
                    C = point_dist(p, facet2D[2])
                    z = (facet[0,2]*A + facet[1,2]*B + facet[2,2]*C)/(A+B+C)
                    
                    if z < self.depth_buffer[y,x]:
                        self.depth_buffer[y,x] = z
                        self.frame_buffer[y,x] = color
    
    def draw_frame_buffer(self):
        self.canvas.put_image_data(self.frame_buffer[:,:,:3], 0, 0)
    
    
    def _draw_axis(self):
        xaxis_obj = Object3D(vertices=np.array([
            [-1000, 0, 0, 1],
            [1000, 0, 0, 1]
        ]), facets=np.array([
            [0,1]
        ]))
        yaxis_obj = Object3D(vertices=np.array([
            [0, -1000, 0, 1],
            [0, 1000, 0, 1],
        ]), facets=np.array([
            [0,1]
        ]))
        zaxis_obj = Object3D(vertices=np.array([
            [0, 0, -50, 1],
            [0, 0, 0, 1],
            [0, 0, 50, 1]
        ]), facets=np.array([
            [0,1],
            [1,2]
        ]))
        old_stroke_style = self.canvas.stroke_style
        old_line_width = self.canvas.line_width
        self.canvas.line_width = 2
        self.canvas.stroke_style = 'red'
        self._draw_obj(xaxis_obj, clip=False)
        self.canvas.stroke_style = 'green'
        self._draw_obj(yaxis_obj, clip=False)
        self.canvas.stroke_style = 'blue'
        self._draw_obj(zaxis_obj, clip=False)
        self.canvas.stroke_style = old_stroke_style
        self.canvas.line_width = old_line_width
    
    def _render(self):
        with hold_canvas():
            if self.show_axis:
                self._draw_axis()
            for obj in self.objects:
                self._draw_obj(obj)
            self.draw_frame_buffer()
    
    def show(self):
        display(self.canvas)
        self._render()
        
    
    def refresh(self):
        self.canvas.clear()
        self.frame_buffer[:,:]=[255,255,255]
        self.depth_buffer[:,:]=1E9*1.0
        self._render()

## 创建场景

In [40]:
camera = Camera([-5,6,-300], math.radians(0),math.radians(0),math.radians(0))
render = Render(height=400, width=800, camera=camera)
render.show_axis = True

## 添加对象

In [41]:
obj1 = Object3D(vertices=np.array([
    [-10, -10, -50, 1],
    [-10,  10, -50, 1],
    [ 10,  10, -50, 1],
    [ 10, -10, -50, 1],
    [-50, -50, 50, 1], 
    [-50,  50, 50, 1], 
    [ 50,  50, 50, 1], 
    [ 50, -50, 50, 1]] 
), facets=np.array([
    [0,1,2,0],
    [0,2,3,0],
    [4,5,6,4],
    [4,5,7,4],
    [0,1,5,0],
    [0,5,4,0],
    [3,2,6,3],
    [3,6,7,3],
    [3,0,4,3],
    [3,4,7,3],
    [1,2,6,1],
    [1,6,5,1]
]), facet_colors=np.array([
    hex_to_rgb("#d11d4d"),
    hex_to_rgb("#921dd1"),
    hex_to_rgb("#1d65d1"),
    hex_to_rgb("#1dd1bc"),
    hex_to_rgb("#1dd150"),
    hex_to_rgb("#eae43f"),
    hex_to_rgb("#ea9d3f"),
    hex_to_rgb("#3f61ea"),
    hex_to_rgb("#ea3fce"),
    hex_to_rgb("#50ea3f"),
    hex_to_rgb("#181a1a"),
    hex_to_rgb("#80f2f0"),
    hex_to_rgb("#d11d1d")
]))

render.add_object(obj1)

## 可视化

In [42]:
render.show()

headingSlider = widgets.FloatSlider(value=math.degrees(render.camera.heading), min=-360, max=360, description='heading:', layout=widgets.Layout(width='500px'))
def headingChanged(evt):
    render.camera.heading = math.radians(evt.new)
    render.refresh()
headingSlider.observe(headingChanged, 'value')

pitchSlider = widgets.FloatSlider(value=math.degrees(render.camera.pitch), min=-360, max=360, description='pitch:', layout=widgets.Layout(width='500px'))
def pitchChanged(evt):
    render.camera.pitch = math.radians(evt.new)
    render.refresh()
pitchSlider.observe(pitchChanged, 'value')

yawSlider = widgets.FloatSlider(value=math.degrees(render.camera.yaw), min=-360, max=360, description='yaw:', layout=widgets.Layout(width='500px'))
def yewChanged(evt):
    render.camera.yaw = math.radians(evt.new)
    render.refresh()
yawSlider.observe(yewChanged, 'value')

xSlider = widgets.FloatSlider(value=render.camera.position[0], min=-400, max=400, description='x:', layout=widgets.Layout(width='500px'))
def xChanged(evt):
    render.camera.position[0] = evt.new
    render.refresh()
xSlider.observe(xChanged, 'value')

ySlider = widgets.FloatSlider(value=render.camera.position[1], min=-400, max=400, description='y:', layout=widgets.Layout(width='500px'))
def yChanged(evt):
    render.camera.position[1] = evt.new
    render.refresh()
ySlider.observe(yChanged, 'value')

zSlider = widgets.FloatSlider(value=render.camera.position[2], min=-400, max=400, description='z:', layout=widgets.Layout(width='500px'))
def zChanged(evt):
    render.camera.position[2] = evt.new
    render.refresh()
zSlider.observe(zChanged, 'value')

widgets.VBox([headingSlider, pitchSlider, yawSlider, xSlider, ySlider, zSlider])

Canvas(height=400, width=800)